# Load Packages

In [1]:
using StatsBase

include("printmat.jl")

println4Ps (generic function with 1 method)

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# Loading Daily S&P 500 Data

In [3]:
xx = readdlm("Data/SP500RfPs.csv",',',header=true)
x  = xx[1]
SP = convert(Array{Float64},x[:,2])      #convert to numerical array, S&P 500 level
R  = (SP[2:end]./SP[1:end-1] - 1) * 100  #returns, % 
T  = length(R)  

dN = Date.(string.(x[2:end,1]),"d/m/y")    #convert to Date, 2:end as for R
println("Number of days in the sample: ",T)

Number of days in the sample: 9352


# Autocorrelations

$\rho_s = \text{Corr}(R_t,R_{t-s})$

In large samples, $\sqrt{T}\hat{\rho}_{s}\sim N(0,1)$, if the true value is $\rho_s=0$ for all $s$

In [4]:
plags = 1:5
ρ = autocor(R,plags)
println("\nlag, autocorrr, t-stat(autocorr) of excess returns")
printmat([plags ρ sqrt(T)*ρ])


lag, autocorrr, t-stat(autocorr) of excess returns
     1.000    -0.026    -2.480
     2.000    -0.044    -4.208
     3.000    -0.007    -0.684
     4.000    -0.022    -2.124
     5.000    -0.018    -1.744



# Autoregressions

An AR(1) is 

$R_{t}=c+a_{1}R_{t-1}+\varepsilon_{t}$

and can be estimated by OLS

An asymmteric AR(1) 

$R_{t} =\alpha+\beta Q_{t-1}R_{t-1}+\gamma(1-Q_{t-1})R_{t-1}+\varepsilon
_{t}$,

where $Q_{t-1}=1 \ \text{ if } \ R_{t-1}<0$ and zero otherwise

In [5]:
function OlsFn(y,x)                      #OLS estimation, alternative: use GLM.jl 
    b     = x\y
    yhat  = x*b
    res   = y - yhat
    Covres = cov(res)
    Covb   = kron(Covres,inv(x'x))       #works also if y is Txn
    R2a    = 1 - var(res,1)./var(y,1)
    return b,res,yhat,Covb,R2            #the outputs
end

OlsFn (generic function with 1 method)

In [6]:
y  = R[2:end]
Tb = size(y,1)
x  = [ones(Tb) R[1:end-1]]
(b,res,yhat,Covb,) = OlsFn(y,x) 
Stdb  = sqrt.(diag(Covb))
tstat = b./Stdb 

println("Results from AR(1): intercept and slope with t-stats")
printmat([b tstat])

Results from AR(1): intercept and slope with t-stats
     0.039     3.405
    -0.026    -2.481



In [7]:
DummyNegative = R[1:end-1] .< 0

x = [ones(Tb) DummyNegative.*R[1:end-1] (1-DummyNegative).*R[1:end-1]]

(b,res,yhat,Covb,) = OlsFn(y,x) 
Stdb  = sqrt.(diag(Covb))
tstat = b./Stdb 

println("Results from AR(1) with dummies: [intercept, slope neg, slope pos] with t-stats")
printmat([b tstat])

Results from AR(1) with dummies: [intercept, slope neg, slope pos] with t-stats
     0.001     0.058
    -0.075    -4.395
     0.027     1.518



## Recursive Estimation and Out-of-Sample R2

Do recursive estimation (longer and longer sample) an predict one period ahead (outside of the sample). Define an "out-of-sample" $R^2$ as 

$R_{OS}^2 = 1- \frac{\text{MSE(forecasting model)}}{\text{MSE(benchmark forecast)}}$

In [9]:
y = R[2:end]
Tb = size(y,1)
x = [ones(Tb) R[1:end-1]]

bT     = fill(NaN,size(x))
rHat   = fill(NaN,Tb)
rTilde = fill(NaN,Tb)
for t = 100:Tb
    (b,res,yhat,Covb,) = OlsFn(y[1:t-1],x[1:t-1,:]) 
    bT[t,:] = b
    rHat[t] = x[t,:]'b                  #out-of-sample forecast of period t
    rTilde[t] = mean(y[1:t-1])          #benchmark forecast: historical average
end    

MSE_Model = mean((y[100:end]-rHat[100:end]).^2)    #MSE for out-of-sample period
MSE_Bench = mean((y[100:end]-rTilde[100:end]).^2)

printlnPs("MSE of AR(1) model and the benchmark: ",[MSE_Model MSE_Bench])
printlnPs("out-of-sample R2: ",1-MSE_Model/MSE_Bench)

MSE of AR(1) model and the benchmark:      1.247     1.244
out-of-sample R2:     -0.002


In [10]:
xTicksLoc = Dates.value.([Date(1980);Date(1990);Date(2000);Date(2010)])
xTicksLab = ["1980";"1990";"2000";"2010"]               #crude way of getting tick marks right

plot(dN[2:end],bT[:,1],color=:blue,legend=false,xticks=(xTicksLoc,xTicksLab))
title!("Estimated intercept")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1980 
 
 
 1990 
 
 
 2000 
 
 
 2010 
 
 
 0.01 
 
 
 0.02 
 
 
 0.03 
 
 
 0.04 
 
 
 0.05 
 
 
 0.06 
 
 
 0.07 
 
 
 Estimated intercept 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 44.4511,283.271 44.6111,269.511 44.6511,280.164 44.6911,329.252 44.7311,328.043 44.8511,324.043 44.8911,317.596 44.9311,255.484 44.9711,232.019 45.0111,214.566 
 45.1311,231.444 45.1711,211.44 45.2111,173.592 45.2511,200.141 45.2911,205.144 45.411,211.402 45.451,236.418 45.491,234.807 45.531,234.066 45.571,215.776 
 45.691,196.558 45.731,221.638 45.771,240.408 45.811,213.298 45.851,196.31 45.971,195.399 46.011,234.083 46.091,227.497 46.131,215.654 46.251,172.815 
 46.291,150.201 46.3309,163.925 46.3709,186.964 46.4109,225.265 46.5309,237.801 46.5709,220.698 46.6109,256.104 46.6509,258.908 46.6909,262.352 46.8109,255.151 
 46.8509,265.232 46.8909,251.057 46.9309,214.935 46.9709,219.367 47.0909,220.673 47.1309,220.133 47.1709,198.704 47.2109,190.961 47.2509,195.887 47.3708,199.028 
 47.4108,191.318 47.4508,150.214 47.4908,147.128 47.5308,164.978 47.6508,133.005 47.6908,109.463 47.7308,111.631 47.7708,91.3867 47.8108,99.724 47.9308,94.4402 
 47.9708,81.4628 48.0108,82.7738 48.0508,82.6248 48.0908,95.3772 48.2107,96.9831 48.2507,82.7223 48.2907,89.2349 48.3307,90.7345 48.3707,92.5802 48.5307,85.7995 
 48.5707,138.845 48.6107,172.036 48.6507,154.174 48.7707,135.528 48.8107,126.945 48.8507,146.888 48.8907,136.406 48.9307,138.036 49.0507,114.778 49.0907,116.875 
 49.1307,140.964 49.1706,130.941 49.2106,76.3673 49.3306,82.4721 49.3706,106.582 49.4106,103.398 49.4506,98.0288 49.4906,94.2439 49.6106,118.175 49.6506,139.092 
 49.6906,108.774 49.7306,112.44 49.7706,99.426 49.8906,77.6188 49.9306,108.857 49.9706,203.823 50.0106,234.642 50.0506,236.075 50.1705,250.161 50.2105,278.09 
 50.2505,277.69 50.2905,272.387 50.3305,268.492 50.4505,319.054 50.4905,334.362 50.5305,341.493 50.5705,335.5 50.6105,347.49 50.7305,331.046 50.7705,330.603 
 50.8105,283.658 50.8505,309.989 50.8905,287.859 51.0105,292.468 51.0505,309.162 51.0904,322.168 51.1304,352.669 51.1704,336.371 51.2904,310.007 51.3304,271.015 
 51.3704,290.369 51.4104,277.901 51.4504,263.792 51.5704,274.279 51.6104,263.442 51.6504,277.207 51.7304,271.115 51.8504,255.372 51.8904,214.797 51.9304,229.32 
 51.9704,219.964 52.0104,221.166 52.1303,235.69 52.1703,241.439 52.2103,220.222 52.2503,215.074 52.2903,202.45 52.4103,214.64 52.4503,210.622 52.4903,215.464 
 52.5303,215.026 52.5703,212.857 52.6903,189.379 52.7303,186.521 52.7703,207.926 52.8103,207.45 52.8503,206.724 52.9703,220.676 53.0502,217.8 53.0902,216.402 
 53.1302,214.025 53.2502,217.547 53.3302,215.956 53.3702,258.229 53.4102,263.503 53.5302,236.128 53.5702,235.186 53.6102,196.936 53.6502,201.208 53.6902,186.855 
 53.8102,189.285 53.8502,181.81 53.8902,170.706 53.9302,174.939 53.9701,181.832 54.0901,174.862 54.1301,159.098 54.1701,171.574 54.2101,135.051 54.2501,136.078 
 54.3701,139.118 54.4101,118.574 54.4501,133.159 54.4901,111.691 54.5301,129.675 54.6501,110.632 54.6901,124.169 54.7301,118.595 54.7701,103.221 54.8101,97.8435 
 54.93,74.6858 54.97,88.4366 55.01,74.5172 55.05,69.0178 55.09,95.9501 55.25,118.73 55.29,131.919 55.33,99.0544 55.37,117.017 55.49,120.402 
 55.53,148.502 55.57,134.505 55.61,159.717 55.65,158.739 55.77,138.751 55.81,156.825 55.8499,151.364 55.8899,177.542 55.9299,220.595 56.0499,249.853 
 56.0899,253.893 56.1299,232.551 56.1699,248.508 56.2099,268.579 56.3299,269.936 56.3699,321.775 56.4099,281.669 56.4499,281.087 56.4899,300.791 56.6099,313.342 
 56.6499,363.491 56.6899,360.201 56.7299,368.739 56.7699,375.813 56.8898,333.491 56.9298,315.441 56.9698,316.552 57.0098,308.951 57.1698,318.307 57.2098,349.439 
 57.2498,329.104 57.2898,301.101 57.3298,290.139 57.4498,296.361 57.4898,311.012 57.5298,312.996 57.5698,329.921 57.6098,

In [11]:
plot(dN[2:end],bT[:,2],color=:red,legend=false,xticks=(xTicksLoc,xTicksLab))
title!("Estimated AR(1) slope")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1980 
 
 
 1990 
 
 
 2000 
 
 
 2010 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 Estimated AR(1) slope 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 44.4511,229.784 44.6111,229.729 44.6511,231.773 44.6911,224.403 44.7311,225.213 44.8511,225.377 44.8911,225.061 44.9311,219.155 44.9711,194.147 45.0111,185.426 
 45.1311,191.16 45.1711,196.039 45.2111,185.206 45.2511,204.682 45.2911,202.685 45.411,201.959 45.451,199.055 45.491,199.701 45.531,199.715 45.571,199.639 
 45.691,193.289 45.731,203.45 45.771,195.517 45.811,204.84 45.851,196.96 45.971,196.604 46.011,198.316 46.091,202.67 46.131,202.166 46.251,192.079 
 46.291,168.857 46.3309,178.057 46.3709,172.624 46.4109,156.319 46.5309,147.978 46.5709,153.099 46.6109,163.604 46.6509,161.776 46.6909,161.314 46.8109,161.961 
 46.8509,163.318 46.8909,166.194 46.9309,156.213 46.9709,160.377 47.0909,160.349 47.1309,160.369 47.1709,160.329 47.2109,156.346 47.2509,157.568 47.3708,157.27 
 47.4108,157.942 47.4508,151.061 47.4908,147.219 47.5308,150.996 47.6508,163.452 47.6908,145.842 47.7308,147.695 47.7708,147.311 47.8108,151.937 47.9308,152.839 
 47.9708,151.523 48.0108,152.009 48.0508,152.009 48.0908,151.988 48.2107,151.491 48.2507,152.658 48.2907,155.19 48.3307,154.966 48.3707,154.853 48.5307,155.268 
 48.5707,164.82 48.6107,127.981 48.6507,140.731 48.7707,134.89 48.8107,129.4 48.8507,136.53 48.8907,141.658 48.9307,141.969 49.0507,142.641 49.0907,144.195 
 49.1307,144.311 49.1706,150.279 49.2106,141.223 49.3306,156.726 49.3706,154.682 49.4106,156.502 49.4506,156.527 49.4906,155.958 49.6106,158.994 49.6506,146.549 
 49.6906,162.546 49.7306,165.533 49.7706,166.211 49.8906,157.609 49.9306,185.12 49.9706,107.271 50.0106,60.5794 50.0506,59.4745 50.1705,56.855 50.2105,46.5077 
 50.2505,46.7839 50.2905,47.4872 50.3305,47.0618 50.4505,53.2541 50.4905,35.5142 50.5305,31.4961 50.5705,33.2207 50.6105,34.3276 50.7305,39.2004 50.7705,39.0349 
 50.8105,35.337 50.8505,72.5288 50.8905,84.0881 51.0105,86.2764 51.0505,85.335 51.0904,79.703 51.1304,67.6646 51.1704,80.8168 51.2904,73.7794 51.3304,42.7451 
 51.3704,69.2311 51.4104,73.5156 51.4504,69.9072 51.5704,74.5253 51.6104,76.8754 51.6504,80.2807 51.7304,82.2855 51.8504,80.7106 51.8904,61.6313 51.9304,83.0193 
 51.9704,85.3981 52.0104,85.6394 52.1303,85.5697 52.1703,83.4117 52.2103,87.8437 52.2503,84.9019 52.2903,81.8197 52.4103,87.014 52.4503,88.1699 52.4903,88.447 
 52.5303,88.5038 52.5703,88.5287 52.6903,87.16 52.7303,84.9302 52.7703,89.3941 52.8103,89.6591 52.8503,89.7216 52.9703,89.7851 53.0502,90.9053 53.0902,90.8882 
 53.1302,90.7894 53.2502,91.061 53.3302,91.2166 53.3702,92.4831 53.4102,86.7261 53.5302,95.5939 53.5702,94.8278 53.6102,89.2861 53.6502,95.4749 53.6902,95.104 
 53.8102,96.2379 53.8502,96.3472 53.8902,93.793 53.9302,95.5394 53.9701,94.9791 54.0901,96.4827 54.1301,93.703 54.1701,101.078 54.2101,113.234 54.2501,114.534 
 54.3701,114.845 54.4101,116.565 54.4501,126.987 54.4901,135.731 54.5301,147.106 54.6501,156.89 54.6901,163.87 54.7301,166.049 54.7701,164.259 54.8101,161.141 
 54.93,155.025 54.97,168.928 55.01,174.687 55.05,172.567 55.09,179.092 55.25,162.238 55.29,154.3 55.33,167.693 55.37,188.034 55.49,186.176 
 55.53,181.943 55.57,192.43 55.61,200.262 55.65,200.96 55.77,201.86 55.81,214.667 55.8499,217.627 55.8899,220.542 55.9299,188.837 56.0499,158.253 
 56.0899,155.055 56.1299,159.495 56.1699,170.123 56.2099,161.254 56.3299,160.449 56.3699,154.727 56.4099,209.878 56.4499,209.321 56.4899,211.045 56.6099,204.115 
 56.6499,184.611 56.6899,189.013 56.7299,188.6 56.7699,186.847 56.8898,196.381 56.9298,174.123 56.9698,174.863 57.0098,174.624 57.1698,176.687 57.2098,169.052 
 57.2498,186.316 57.2898,173.286 57.3298,163.419 57.4498,166.081 57.4898,164.093 57.5298,163.279 57.5698,161.563 57.6098,158.642 57.7298,157.028 57.7697,154.478 
 57.8097,174.577 